In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.graphics.api as smg
import matplotlib.dates as md
import xgboost
from sklearn import datasets, linear_model, svm
from statsmodels.formula.api import ols
from patsy.highlevel import dmatrices
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import scale
from scipy.stats import spearmanr
from pylab import rcParams
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import seaborn as sb
from sklearn import metrics
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic

from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
pd.options.display.max_rows=10
from datetime import datetime
import numpy as np
df = pd.read_table("grenoble_7.k7",sep = ',',header = 0)

class_names=['Bad','Intermediate','Good']

def trace_conf_mat(cm, acc,classes, norm, title,cmap=plt.cm.Blues):
    accuracy=acc
    
    classes = ['Bad','Intermediate','Good']
    if norm:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title+'%.4f' % accuracy,
           ylabel='True label',
           xlabel='Predicted label')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.3f' if norm else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def val_prediction_pdr(df):
    ret=list()
    for i in range(len(df)):
        if df.index.values.astype(int)[i] < 0.02:
            ret.append(0.5)
        elif df.index.values.astype(int)[i] >= 0.02 and df.index.values.astype(int)[i] < 0.25:
            ret.append(df.index.values.astype(int)[i])
        elif df.index.values.astype(int)[i] >= 0.25 and df.index.values.astype(int)[i] < 0.3:
            ret.append(0.9)
        elif df.index.values.astype(int)[i] >= 0.3 and df.index.values.astype(int)[i] <= 0.35:
            ret.append(0.1)
        elif df.index.values.astype(int)[i] > 0.35 and df.index.values.astype(int)[i] <= 0.73:
            ret.append(df.index.values.astype(int)[i])
        elif df.index.values.astype(int)[i] >= 0.73 and df.index.values.astype(int)[i] <= 0.75:
            ret.append(0.9)
        elif df.index.values.astype(int)[i] > 0.75 and df.index.values.astype(int)[i] <= 0.8:
            ret.append(0.1)
        elif df.index.values.astype(int)[i] > 0.8 and df.index.values.astype(int)[i] < 0.85:
            ret.append(0.5)
        else:
            ret.append(df.index.values.astype(int)[i])
    return ret


def logistic_regression(df, classes, normalize=False, title=None,cmap=plt.cm.Blues):
    ros = RandomOverSampler(random_state=0)
    rus = RandomUnderSampler(random_state=0)
    y_true=list()
    y_pred=list()
    target=list()
    val_pred=val_prediction_pdr(df)
    for i in range(len(df)):
        if df.index.values.astype(int)[i] < 0.3:
            target.append(0)
        elif df.index.values.astype(int)[i] > 0.75:
            target.append(2)
        else:
            target.append(1)
        if val_pred[i] < 0.3:
            y_pred.append(0)
        elif val_pred[i] > 0.75:
            y_pred.append(2)
        else:
            y_pred.append(1)
    df['target']=target
    df['pred']=y_pred
    
    mydata=df.ix[:,(5,7)].values#Column 5 corresponds to the value of PDR and column 7 to the value representing that valu in class(BadnGood and Intermediate)
    X=scale(mydata)
    y=df['pdr']
    y=df['pdr']
    #X_resampled, y_resampled = ros.fit_resample(X, y)
    
    
    
    #X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    print(X_train)
    print(y_test,y_train)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    
    y_pred = logreg.predict(X_test)
    
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_pred))
    
    #print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    #Plotting of Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    accuracy=logreg.score(X_test, y_test)
    
    trace_conf_mat(cm, accuracy,classes, normalize, title,cmap=plt.cm.Blues)
np.set_printoptions(precision=2)

#print(df)
ts=df.loc[(df['channel']==16)]
#df=pd.DataFrame(ts.loc[((ts['src']==1)&(ts['dst']==14))])

logistic_regression(df, classes=class_names, normalize=True,title='LR Accuracy of PDR= ')
plt.savefig("PDR\\LRConfusionMatrixWithoutNormalizationPDR.png", format='png', bbox_inches='tight', pad_inches=0)

logistic_regression(df, classes=class_names, title='LR Accuracy of PDR= ') 
plt.savefig("PDR\\LRConfusionMatrixWithoutNormalization.png", format='png', bbox_inches='tight', pad_inches=0)

KeyboardInterrupt: 